In [ ]:
import os
import numpy as np
import pandas as pd
from astropy.table import Table

from lvmdrp import path, __version__ as drpver
from lvmdrp.utils import metadata as md
from lvmdrp.functions import run_drp as drp

from lvmdrp.functions import imageMethod, rssMethod

ORIG_MASTER_DIR = os.getenv("LVM_MASTER_DIR")
MASTER_CON_LAMPS = {"b": "ldls", "r": "ldls", "z": "quartz"}
MASTER_ARC_LAMPS = {"b": "hgne", "r": "neon", "z": "neon"}

In [ ]:
frames_table = md.get_metadata(tileid="*", mjd=60177)
frames_table.query("imagetyp == 'bias' | imagetyp == 'dark' | imagetyp == 'flat' | imagetyp == 'arc'", inplace=True)

In [ ]:
masters_mjd = frames_table.mjd.min()
masters_path = os.path.join(ORIG_MASTER_DIR, f"{masters_mjd}")

In [ ]:
masters_flat = md.get_metadata(kind="master", imagetyp="flat").query("mjd == @masters_mjd").sort_values("camera")
masters_flat

In [ ]:
mtrace_paths = []
for flat in masters_flat.to_dict("records"):
    camera = flat["camera"]
    # define lamps for flats and arcs
    con_lamp = MASTER_CON_LAMPS[camera[0]]
    arc_lamp = MASTER_ARC_LAMPS[camera[0]]
    
    # define masters paths
    mflat_path = path.full("lvm_master", drpver=drpver, tileid=flat["tileid"], mjd=masters_mjd, kind=f"mflat_{con_lamp}", camera=camera)
    mtrace_path = os.path.join(masters_path, f"lvm-mtrace-{camera}.fits")
    mwidth_path = os.path.join(masters_path, f"lvm-mwidth-{camera}.fits")
    mwave_path = os.path.join(masters_path, f"lvm-mwave_{arc_lamp}-{camera}.fits")
    mlsf_path = os.path.join(masters_path, f"lvm-mwave_{arc_lamp}-{camera}.fits")
    # define master fiberflat path
    wflat_path = path.full("lvm_master", drpver=drpver, tileid=flat["tileid"], mjd=masters_mjd, kind=f"wmflat_{con_lamp}", camera=camera)

    if os.path.exists(wflat_path):
        print(f"skipping {wflat_path}, file already exist")
        continue
        
    print(f"Processing {mflat_path} -> {wflat_path}")

    # TODO: run the fiber fitting extraction to remove the aperture extraction features
    imageMethod.extract_spectra(in_image=mflat_path, out_rss=wflat_path, in_trace=mtrace_path, fwhm=mwidth_path, method="optimal", parallel=4)
    # imageMethod.extract_spectra(in_image=mflat_path, out_rss=wflat_path, in_trace=mtrace_path, method="aperture", aperture=3)
    rssMethod.create_pixel_table(in_rss=wflat_path, out_rss=wflat_path, arc_wave=mwave_path, arc_fwhm=mlsf_path)

In [ ]:
# wflat_paths = []
# mfiberflat_paths = []
# for flat in masters_flat.to_dict("records"):
#     camera = flat["camera"]
#     # define lamps for flats and arcs
#     con_lamp = MASTER_CON_LAMPS[camera[0]]
#     arc_lamp = MASTER_ARC_LAMPS[camera[0]]
    
#     # define master fiberflat path
#     wflat_path = path.full("lvm_master", drpver=drpver, tileid=flat["tileid"], mjd=masters_mjd, kind=f"wmflat_{con_lamp}", camera=camera)
#     # wflat_path = wflat_path.replace("lvm-", "lvm_fitting-")
#     wflat_paths.append(wflat_path)
    
#     mfiberflat_path = os.path.join(masters_path, f"lvm-mfiberflat-{camera}.fits")
#     mfiberflat_paths.append(mfiberflat_path)

# rssMethod.create_fiberflat(in_rsss=wflat_paths, out_rsss=mfiberflat_paths, median_box=200, gaussian_kernel=10, display_plots=True)

In [ ]:
for channel in "brz":
    wflat_paths, mfiberflat_paths = [], []
    for flat in masters_flat.to_dict("records"):
        camera = flat["camera"]
        if camera[0] != channel: continue
        con_lamp = MASTER_CON_LAMPS[camera[0]]
        wflat_path = path.full("lvm_master", drpver=drpver, tileid=flat["tileid"], mjd=masters_mjd, kind=f"wmflat_{con_lamp}", camera=camera)
        mfiberflat_path = os.path.join(masters_path, f"lvm-mfiberflat-{camera}.fits")
        mfiberflat_paths.append(mfiberflat_path)
        wflat_paths.append(wflat_path)

    # TODO: smooth extracted spectra before computing the median spectrum
    # rssMethod.create_fiberflat(in_rsss=wflat_paths, out_rsss=mfiberflat_paths, median_box=200, gaussian_kernel=10, display_plots=True)
    rssMethod.create_fiberflat(in_rsss=wflat_paths, out_rsss=mfiberflat_paths, median_box=0, gaussian_kernel=10, display_plots=True)

# cache new masters metadata into HDF5 files
new_masters = md.extract_metadata(frames_paths=mfiberflat_paths)
md.add_masters(new_masters)

In [ ]:
# fibers = [2, 106, 37]
# select = np.isin(fibermap["fiberid"]-1, fibers)
# skw1_fibers = fibermap[~select][fibermap[~select]["ifulabel"] == "SkyW1"]["fiberid"]-1
# ske1_fibers = fibermap[~select][fibermap[~select]["ifulabel"] == "SkyE1"]["fiberid"]-1
# sci1_fibers = fibermap[~select][fibermap[~select]["ifulabel"] == "Sci1"]["fiberid"]-1

# factors = {}
# for spec in "123":
#     for cam in ["b", "r", "z"]:
#         cam = cam + spec
#         fflat = RSS()
#         fflat.loadFitsData(f"/home/mejia/Research/lvm/lvmdata/calib/60177/lvm-mfiberflat-{cam}.fits")
#         fflat._data[(fflat._mask)|(fflat._data <= 0)] = np.nan
#         sci_factor = np.nanmedian(fflat._data[sci1_fibers][:, 1000:3000])
#         print(cam, np.nanmedian(fflat._data[skw1_fibers][:, 1000:3000])/sci_factor, np.nanmedian(fflat._data[ske1_fibers][:, 1000:3000])/sci_factor)
#         factors[("SkyW", cam)] = np.nanmedian(fflat._data[skw1_fibers][:, 1000:3000])/sci_factor
#         factors[("SkyE", cam)] = np.nanmedian(fflat._data[ske1_fibers][:, 1000:3000])/sci_factor